In [1]:
import pyrealsense2 as rs
import matplotlib as plt
import cv2
import logging
import numpy as np
import time

In [2]:
ctx = rs.context()
devices = ctx.query_devices()

In [3]:
var = [i.get_info(rs.camera_info.serial_number) for i in devices]
var

['923322070538', '923322071362']

In [4]:
dev = devices[0].first_depth_sensor()
dev.set_option(rs.option.inter_cam_sync_mode,1)
dev.get_option(rs.option.inter_cam_sync_mode)

1.0

In [5]:
dev2 = devices[1].first_depth_sensor()
dev2.set_option(rs.option.inter_cam_sync_mode,2)
dev2.get_option(rs.option.inter_cam_sync_mode)

2.0

In [6]:
#cam1: configure depth and color streams
pipeline1 = rs.pipeline()
config1 = rs.config()
config1.enable_device(var[0])
config1.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)
config1.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
print('cam1 enabled')

#cam2: configure depth and color streams 
pipeline2 = rs.pipeline()
config2 = rs.config()
config2.enable_device(var[1])
config2.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)
config2.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
print('cam2 enabled')

cam1 enabled
cam2 enabled


In [35]:
#set camera to master
#pipe = rs.pipeline()
#prof = pipe.start()
#dev = prof.get_device()
#ds = dev.query_sensors()[0]
#ds.set_option(rs.option.inter_cam_sync_mode,1)
#ds.get_option(rs.option.inter_cam_sync_mode)

In [7]:
pipeline1.start(config1)
pipeline2.start(config2)
print('cams configued')

cams configued


In [8]:
try:
    start = time.time()
    while time.time() - start <10:
        
        #camera 1
        frames1 = pipeline1.wait_for_frames()
        depthframe1 = frames1.get_depth_frame()
        colframe1 = frames1.get_color_frame()
        if not depthframe1 or not colframe1:
            continue

        #convert images to numpy arrays
        depthim1 = np.asanyarray(depthframe1.get_data())
        colim1 = np.asanyarray(colframe1.get_data())
        
        #apply color map on depth image, convert to 8bit pixel
        depthcolmap1 = cv2.applyColorMap(cv2.convertScaleAbs(depthim1, alpha=0.5), cv2.COLORMAP_JET)
        
        #camera 2
        frames2 = pipeline2.wait_for_frames()
        depthframe2 = frames2.get_depth_frame()
        colframe2 = frames2.get_color_frame()
        if not depthframe2 or not colframe2:
            continue

            
        #check time sync
        print([depthframe1, depthframe2]) #returns system_time
        #counter = rs.frame.get_frame_metadata(depthframe1, rs.frame_metadata_value.frame_counter)
        #counter2 = rs.frame.get_frame_metadata(depthframe2, rs.frame_metadata_value.frame_counter)
        #print([counter,counter2])
        #convert images to numpy arrays
        depthim2 = np.asanyarray(depthframe2.get_data())
        colim2 = np.asanyarray(colframe2.get_data())
        
        #apply color map on depth image, convert to 8bit pixel
        depthcolmap2 = cv2.applyColorMap(cv2.convertScaleAbs(depthim2, alpha=0.5), cv2.COLORMAP_JET)
        
        
        #stack all images horizontally
        images = np.hstack((colim1, depthcolmap1, colim2, depthcolmap2))
        
        #show images from cameras
        cv2.namedWindow('RealSense', cv2.WINDOW_NORMAL)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
    
finally:
    #stop streaming
    pipeline1.stop()
    pipeline2.stop()
    cv2.destroyAllWindows()
    del pipeline1
    del config1
    del pipeline2
    del config2

[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71294470>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712942f0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71294430>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712830b0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71283df0>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71294530>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712947f0>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712944f0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c732fdd30>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712945f0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712830b0>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71283df0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712948b0>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712942f0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71294870>, <p

[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712830b0>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71294630>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712944b0>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c732fdd30>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71283df0>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712945b0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71294670>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712830b0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71294870>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712946b0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c732fdd30>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c71283df0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712946f0>, <pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712944f0>]
[<pyrealsense2.pyrealsense2.depth_frame object at 0x7f5c712830b0>, <p

NameError: name 'config1' is not defined